# 🫔 Notion MCP Agent

# 1. ⚙️ Setup

In [46]:
from utilities.setup import *

In [47]:
GOOGLE_API_KEY = get_google_api_key()
retry_config = get_retry_config()
NOTION_API_KEY = get_notion_credentials()

✅ Gemini API key setup complete.
✅ Notion API key setup complete.


# 2. 🧑‍💻 Define Kaggle MCP Agent

In [48]:
mcp_notion_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "@notionhq/notion-mcp-server"
            ],
            env={
                "NOTION_TOKEN": NOTION_API_KEY,
            }
        ),
        timeout=60
    )
)

print("✅ MCP Notion Toolset defined.")

✅ MCP Notion Toolset defined.


In [49]:
notion_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="notion_agent",
    instruction=(
        "You are a Notion assistant. "
        "CRITICAL INSTRUCTIONS FOR READING CONTENT: "
        "1. To read a page, use the tool 'API-get-block-children'. "
        "2. The tool input argument is 'block_id' (pass the page_id here). "
        "3. **DATA EXTRACTION RULE**: The output is a JSON list of blocks. "
        "   Text is hidden deep inside. You must iterate through 'results', "
        "   check fields like 'paragraph', 'heading_1', 'bulleted_list_item', etc., "
        "   and extract the 'plain_text' from within the 'rich_text' array. "
        "   Do not say the page is empty just because you see JSON wrappers. Dig for the 'plain_text'."
        "4. If 'API-post-search' returns nothing, ask the user to 'Connect' the page to the integration."
    ),
    tools=[mcp_notion_server],
)

In [50]:
runner = InMemoryRunner(agent=notion_agent)

In [51]:
# 1. Create a runner specifically for the Notion Agent
# (Assuming you are using the same Runner class as before, likely 'McpRunner' or similar from your imports)
# notion_runner = McpRunner(notion_agent)

# 2. Now run the debug command on the *notion_runner*
# Notice we do NOT pass 'agent=' here anymore.
response = await runner.run_debug(
    "Read the content of page ID '2b5c2851a6528022b0fac4f022403865' using API-get-block-children. Extract and summarize the text.",
    verbose=True
)


 ### Created new session: debug_session_id

User > Read the content of page ID '2b5c2851a6528022b0fac4f022403865' using API-get-block-children. Extract and summarize the text.


/Users/myte/Documents/Projects/Note.Tea/venv/lib/python3.11/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:103: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


notion_agent > [Calling tool: API-get-block-children({'block_id': '2b5c2851a6528022b0fac4f022403865'})]
notion_agent > [Tool result: {'content': [{'type': 'text', 'text': '{"object":"list","results":[{"object":"block","id":"2b5c2851-...]
notion_agent > The content of the page includes the following:
- A to-do item: "Create an account with Notion" (checked)
- A to-do item with instructions on how to use slash commands to create different block types.
- A to-do item instructing the user to use the sidebar to find, organize, and add new pages.
- A to-do item mentioning a "Todo List" on the left with more Notion tips.
- A toggle block with the text "This is a toggle block. Click the little triangle to see a few more useful tips!"
- An empty paragraph block.


In [53]:
response

[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'block_id': '2b5c2851a6528022b0fac4f022403865'
         },
         id='adk-8ce09857-99a5-4848-a1ab-67c85c595932',
         name='API-get-block-children'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=50,
   prompt_token_count=5447,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=5447
     ),
   ],
   total_token_count=5497
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-073f365

🎉 Completed the **🕵🏻‍♂️ Notion MCP Agent**.